In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, IntegerType, ArrayType
from pyspark.sql.functions import col, lit, array, explode, map_keys, map_values
from pyspark.sql import Row

spark = SparkSession.builder.appName("show").getOrCreate()

In [2]:
from pyspark.sql.functions import when

In [3]:
data = [(1, 'Maheer', 2000,500),
       (2, 'wafa', 4000, 1000)]

schema = ['id', 'name', 'salary', 'bonus']

df = spark.createDataFrame(data, schema)

df.show()

+---+------+------+-----+
| id|  name|salary|bonus|
+---+------+------+-----+
|  1|Maheer|  2000|  500|
|  2|  wafa|  4000| 1000|
+---+------+------+-----+



In [4]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- bonus: long (nullable = true)



In [5]:
def totalPay(s,b):
    return s*b

In [6]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType

In [7]:
TotalPay = udf(lambda x,y: totalPay(x,y), IntegerType())

In [8]:
df.select('*', TotalPay(col('salary'), col('bonus')).alias('totalPay')).show()

+---+------+------+-----+--------+
| id|  name|salary|bonus|totalPay|
+---+------+------+-----+--------+
|  1|Maheer|  2000|  500| 1000000|
|  2|  wafa|  4000| 1000| 4000000|
+---+------+------+-----+--------+



In [9]:
df.withColumn('toPay', TotalPay(df.salary, df.bonus)).show()

+---+------+------+-----+-------+
| id|  name|salary|bonus|  toPay|
+---+------+------+-----+-------+
|  1|Maheer|  2000|  500|1000000|
|  2|  wafa|  4000| 1000|4000000|
+---+------+------+-----+-------+



In [10]:
@udf(returnType=IntegerType())
def totalPays(s,b):
    return s+b

In [11]:
df.select('*', totalPays(df.salary, df.bonus)).show()

+---+------+------+-----+------------------------+
| id|  name|salary|bonus|totalPays(salary, bonus)|
+---+------+------+-----+------------------------+
|  1|Maheer|  2000|  500|                    2500|
|  2|  wafa|  4000| 1000|                    5000|
+---+------+------+-----+------------------------+



In [13]:
df.createOrReplaceTempView('emps')

In [15]:
def totalPayss(s,b):
    return s+b

In [16]:
spark.udf.register(name='TotalPaySQL', f=totalPayss, returnType=IntegerType())

<function __main__.totalPayss(s, b)>

In [21]:
df.select('SELECT * FROM emps').show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `SELECT * FROM emps` cannot be resolved. Did you mean one of the following? [`bonus`, `name`, `id`, `salary`].;
'Project ['SELECT * FROM emps]
+- LogicalRDD [id#0L, name#1, salary#2L, bonus#3L], false


In [18]:
df.select('SELECT id, name TotalPaySQL(salary, bonus) as totPay FROM emps')

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `SELECT id, name TotalPaySQL(salary, bonus) as totPay FROM emps` cannot be resolved. Did you mean one of the following? [`salary`, `bonus`, `name`, `id`].;
'Project ['SELECT id, name TotalPaySQL(salary, bonus) as totPay FROM emps]
+- LogicalRDD [id#0L, name#1, salary#2L, bonus#3L], false
